In [8]:
# Prep
import pandas as pd
from datetime import datetime
import re
import os

data_dir = 'C:/Users/kgk/OneDrive - Aalborg Universitet/Undervisning/E22/sds/datasets'
parl_dir = os.path.join(data_dir, 'dk_parl')
eurob_path = os.path.join(data_dir, 'eurobarometer-96_dk_subset.csv')
parl_path = os.path.join(parl_dir, 'dkparl_125_20220825.txt')

with open(parl_path, 'r', encoding = 'utf-8') as f:
    parl_ref = f.read()
    
eurob = pd.read_csv(eurob_path)

## Brug af egne funktioner

Fordelen ved at skrive egne funktioner er, at man kan lave funktioner, der er helt specifikke til de datasæt, man sidder med, og de datahåndtering, som man har brug for.

Egne funktioner kan have flere anvendelser, fx:
- Specifik datahåndteringsfunktion, som skal bruges på flere kolonner/datasæt (fx med mange betingelser)
- Behandling af bestemt type fil (fx tekstfil)
- Konvertering fra en datastruktur til en anden

### Funktion til at behandle tekst

Tidligere arbejdede vi lidt med et referat fra et møde i Folketinget. Strukturen i dokumentet tillader, at vi kan dele det op i dagsordenspunkter:

In [16]:
import re

point_pattern = re.compile("---+")

parlref_points = point_pattern.split(parl_ref)

parlref_points[2]

'\n\nDet første punkt på dagsordenen er:\n\n1) Indstilling fra Udvalget til Valgs Prøvelse:\n\nGodkendelse af stedfortrædere som ordinære medlemmer af Folketinget for\nKristian Thulesen Dahl (UFG) og Rune Lund (UFG).\n\nKl. 13:00\n\nForhandling\n\nFørste næstformand (Karen Ellemann):\n\nFra Udvalget til Valgs Prøvelse har jeg modtaget indstilling om, at 1.\nstedfortræder for Dansk Folkeparti i Sydjyllands Storkreds, Susanne\nEilersen, godkendes som ordinært medlem af Folketinget fra og med den 1.\naugust 2022, da Kristian Thulesen Dahl nedlagde sit mandat pr. 31. juli\n2022.\n\nJeg har endvidere modtaget indstilling om, at 1. stedfortræder for\nEnhedslisten i Københavns Storkreds, Jonathan Simmel, godkendes som\nordinært medlem af Folketinget fra og med den 12. august 2022, da Rune\nLund nedlagde sit mandat pr. 11. august 2022.\n\nEr der nogen, der ønsker ordet?\n\nDa det ikke er tilfældet, går vi til afstemning.\n\nKl. 13:01\n\nAfstemning\n\nFørste næstformand (Karen Ellemann):\n\nDer

Derfra kunne vi være interesseret i at udlede titlerne for punkterne. For at gøre dette laver laver vi en dictionary for hvert punkt (de to elementer i listen er ikke dagsordenspunkter).

In [28]:
point_t_re = re.compile("(?<=\n)\d\) .*?(?=:?\n)")

parlref_pointsdata = []

for point in parlref_points[2:len(parlref_points)]:
    if point_t_re.search(point):
        point_t = point_t_re.search(point).group(0)
    else:
        point_t = ""
    
    point_dict = {"title": point_t, "text": point}
    
    parlref_pointsdata.append(point_dict)

In [29]:
parlref_pointsdata[0]

{'title': '1) Indstilling fra Udvalget til Valgs Prøvelse',
 'text': '\n\nDet første punkt på dagsordenen er:\n\n1) Indstilling fra Udvalget til Valgs Prøvelse:\n\nGodkendelse af stedfortrædere som ordinære medlemmer af Folketinget for\nKristian Thulesen Dahl (UFG) og Rune Lund (UFG).\n\nKl. 13:00\n\nForhandling\n\nFørste næstformand (Karen Ellemann):\n\nFra Udvalget til Valgs Prøvelse har jeg modtaget indstilling om, at 1.\nstedfortræder for Dansk Folkeparti i Sydjyllands Storkreds, Susanne\nEilersen, godkendes som ordinært medlem af Folketinget fra og med den 1.\naugust 2022, da Kristian Thulesen Dahl nedlagde sit mandat pr. 31. juli\n2022.\n\nJeg har endvidere modtaget indstilling om, at 1. stedfortræder for\nEnhedslisten i Københavns Storkreds, Jonathan Simmel, godkendes som\nordinært medlem af Folketinget fra og med den 12. august 2022, da Rune\nLund nedlagde sit mandat pr. 11. august 2022.\n\nEr der nogen, der ønsker ordet?\n\nDa det ikke er tilfældet, går vi til afstemning.\n\nK

In [30]:
[point.get('title') for point in parlref_pointsdata]

['1) Indstilling fra Udvalget til Valgs Prøvelse',
 '2) 1. behandling af lovforslag nr. L 204',
 '3) 1. behandling af lovforslag nr. L 205',
 '4) 1. behandling af lovforslag nr. L 206',
 '5) Valg af 9 medlemmer til DIPD (Bestyrelsen for Dansk Institut for',
 '',
 '']

Ovenstående kunne også laves til en funktion. Med denne har man en funktion, der kan anvendes på referater af møder i Folketinget generelt:

In [33]:
def create_parlpointsdata(parlreftext):
    
    #regex
    point_pattern = re.compile("---+")
    point_t_re = re.compile("(?<=\n)\d\) .*?(?=:?\n)")
    
    #split into points
    parlref_points = point_pattern.split(parlreftext)
    
    #create dataset (key for title and text)
    parlref_pointsdata = []

    for point in parlref_points[2:len(parlref_points)]:
        if point_t_re.search(point):
            point_t = point_t_re.search(point).group(0)
        else:
            point_t = ""

        point_dict = {"title": point_t, "text": point}

        parlref_pointsdata.append(point_dict)
        
    return(parlref_pointsdata)

In [34]:
parlref_points_test = create_parlpointsdata(parl_ref)

parlref_points_test[0]

{'title': '1) Indstilling fra Udvalget til Valgs Prøvelse',
 'text': '\n\nDet første punkt på dagsordenen er:\n\n1) Indstilling fra Udvalget til Valgs Prøvelse:\n\nGodkendelse af stedfortrædere som ordinære medlemmer af Folketinget for\nKristian Thulesen Dahl (UFG) og Rune Lund (UFG).\n\nKl. 13:00\n\nForhandling\n\nFørste næstformand (Karen Ellemann):\n\nFra Udvalget til Valgs Prøvelse har jeg modtaget indstilling om, at 1.\nstedfortræder for Dansk Folkeparti i Sydjyllands Storkreds, Susanne\nEilersen, godkendes som ordinært medlem af Folketinget fra og med den 1.\naugust 2022, da Kristian Thulesen Dahl nedlagde sit mandat pr. 31. juli\n2022.\n\nJeg har endvidere modtaget indstilling om, at 1. stedfortræder for\nEnhedslisten i Københavns Storkreds, Jonathan Simmel, godkendes som\nordinært medlem af Folketinget fra og med den 12. august 2022, da Rune\nLund nedlagde sit mandat pr. 11. august 2022.\n\nEr der nogen, der ønsker ordet?\n\nDa det ikke er tilfældet, går vi til afstemning.\n\nK

### Brug af egne funktioner i pandas

Pandas tillader også, at man kan anvende sine egne funktioner. Dette gøres med metoden `apply()`.

`.apply()` tager en funktion som input. Denne funktion skal gerne virke med en enkelt værdi som input (en værdi fra en kolonne), da `.apply()` så sørger for at anvende den på alle værdierne i kolonnen.

In [35]:
eurob.head()

,uniqid,d11,polintr,qb1,qb3_1,qb3_2,qb3_3,qb3_4,qb3_5,qb3_6,...,d10,d15a,d15b,d25,d63,d1,p1,p2,p3,region_denmark
0,110005573,71,Not at all,Fairly important,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,Man,"Retired, unable to work","Middle management, etc.",Large town,The middle class of society,9,16 Sep 21,13 - 16 h,2099,DK05 - Nordjylland
1,110005574,23,Medium,Very important,Use of personal data and information by compan...,Cyber-attacks and cybercrime such as theft or ...,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,Man,Student,"Unskilled manual worker, etc.",Large town,The middle class of society,6,16 Sep 21,17 - 19 h,2784,DK05 - Nordjylland
2,110005575,53,Medium,Very important,Not mentioned,Cyber-attacks and cybercrime such as theft or ...,The difficulty of learning new digital skills ...,Not mentioned,Not mentioned,Not mentioned,...,Woman,"Unskilled manual worker, etc.",Inap. (not 1 to 4 in d15a),Rural area or village,The middle class of society,8,17 Sep 21,13 - 16 h,1738,DK02 - Sjaelland
3,110005576,70,Medium,Very important,Not mentioned,Cyber-attacks and cybercrime such as theft or ...,Not mentioned,The safety and well-being of children,The difficulty some people have accessing the ...,Not mentioned,...,Man,"Retired, unable to work","Middle management, etc.",Rural area or village,The middle class of society,8,17 Sep 21,13 - 16 h,2759,DK02 - Sjaelland
4,110005577,71,Medium,Fairly important,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,Woman,"Employed position, service job",Inap. (not 1 to 4 in d15a),Small/middle town,The working class of society,5,17 Sep 21,8 - 12 h,1768,DK01 - Hovedstaden


In [40]:
def replacer(cat):
    if cat == "Not mentioned":
        value = 0
    else:
        value = 1
        
    return(value)
        
eurob['qb3_1_bin'] = eurob['qb3_1'].apply(replacer)

eurob.head()

,uniqid,d11,polintr,qb1,qb3_1,qb3_2,qb3_3,qb3_4,qb3_5,qb3_6,...,d15a,d15b,d25,d63,d1,p1,p2,p3,region_denmark,qb3_1_bin
0,110005573,71,Not at all,Fairly important,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,"Retired, unable to work","Middle management, etc.",Large town,The middle class of society,9,16 Sep 21,13 - 16 h,2099,DK05 - Nordjylland,0
1,110005574,23,Medium,Very important,Use of personal data and information by compan...,Cyber-attacks and cybercrime such as theft or ...,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,Student,"Unskilled manual worker, etc.",Large town,The middle class of society,6,16 Sep 21,17 - 19 h,2784,DK05 - Nordjylland,1
2,110005575,53,Medium,Very important,Not mentioned,Cyber-attacks and cybercrime such as theft or ...,The difficulty of learning new digital skills ...,Not mentioned,Not mentioned,Not mentioned,...,"Unskilled manual worker, etc.",Inap. (not 1 to 4 in d15a),Rural area or village,The middle class of society,8,17 Sep 21,13 - 16 h,1738,DK02 - Sjaelland,0
3,110005576,70,Medium,Very important,Not mentioned,Cyber-attacks and cybercrime such as theft or ...,Not mentioned,The safety and well-being of children,The difficulty some people have accessing the ...,Not mentioned,...,"Retired, unable to work","Middle management, etc.",Rural area or village,The middle class of society,8,17 Sep 21,13 - 16 h,2759,DK02 - Sjaelland,0
4,110005577,71,Medium,Fairly important,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,...,"Employed position, service job",Inap. (not 1 to 4 in d15a),Small/middle town,The working class of society,5,17 Sep 21,8 - 12 h,1768,DK01 - Hovedstaden,0
